In [1]:
import ast
import logging
import kagglehub
import pandas as pd
from pathlib import Path

/Users/wennanshi/VScodeProjects/Text_Sentiment_Classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s][%(name)s]: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

In [3]:
path = Path(kagglehub.dataset_download("oliviervha/crypto-news"))
file = path / "cryptonews.csv"
df = pd.read_csv(file)

In [4]:
df.head()

,date,sentiment,source,subject,text,title,url
0,2023-12-19 06:40:41,"{'class': 'negative', 'polarity': -0.1, 'subje...",CryptoNews,altcoin,Grayscale CEO Michael Sonnenshein believes the...,Grayscale CEO Calls for Simultaneous Approval ...,https://cryptonews.comhttps://cryptonews.com/n...
1,2023-12-19 06:03:24,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,"In an exclusive interview with CryptoNews, Man...",Indian Government is Actively Collaborating Wi...,https://cryptonews.comhttps://cryptonews.com/n...
2,2023-12-19 05:55:14,"{'class': 'positive', 'polarity': 0.05, 'subje...",CryptoNews,blockchain,According to the Federal Court ruling on Decem...,Judge Approves Settlement: Binance to Pay $1.5...,https://cryptonews.comhttps://cryptonews.com/n...
3,2023-12-19 05:35:26,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CoinTelegraph,blockchain,Some suggest EVM inscriptions are the latest w...,Why a gold rush for inscriptions has broken ha...,https://cointelegraph.com/news/inscriptions-ev...
4,2023-12-19 05:31:08,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CoinTelegraph,ethereum,A decision by bloXroute Labs to start censorin...,‘Concerning precedent’ — bloXroute Labs' MEV r...,https://cointelegraph.com/news/concerning-prec...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31037 entries, 0 to 31036
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       31037 non-null  object
 1   sentiment  31037 non-null  object
 2   source     31037 non-null  object
 3   subject    31037 non-null  object
 4   text       31037 non-null  object
 5   title      31037 non-null  object
 6   url        31037 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


In [6]:
df.columns

Index(['date', 'sentiment', 'source', 'subject', 'text', 'title', 'url'], dtype='object')

In [7]:
df["date_raw"] = df["date"]
df["date"] = pd.to_datetime(df["date"], errors="coerce")

In [8]:
df["date"].min(), df["date"].max()

(Timestamp('2021-10-12 20:00:00'), Timestamp('2023-12-19 06:40:41'))

In [9]:
df.loc[df["date"].isna(), "date_raw"]

30606    2022-01-12 00:00
30619    2022-01-11 00:00
Name: date_raw, dtype: object

In [10]:
df[["date", "sentiment", "source", "subject"]].isna().any(axis=0)

date          True
sentiment    False
source       False
subject      False
dtype: bool

In [11]:
missing_text_title = df["text"].isna() & df["title"].isna()

In [12]:
def parse_sentiment(x):
    if not isinstance(x, str):
        return {}
    try:
        field = ast.literal_eval(x)
        return field if isinstance(field, dict) else {}
    except Exception:
        return {}

In [13]:
sentiment = df["sentiment"].apply(parse_sentiment)
sentiment

0        {'class': 'negative', 'polarity': -0.1, 'subje...
1        {'class': 'neutral', 'polarity': 0.0, 'subject...
2        {'class': 'positive', 'polarity': 0.05, 'subje...
3        {'class': 'positive', 'polarity': 0.5, 'subjec...
4        {'class': 'neutral', 'polarity': 0.0, 'subject...
                               ...                        
31032    {'class': 'neutral', 'polarity': 0.0, 'subject...
31033    {'class': 'positive', 'polarity': 0.1, 'subjec...
31034    {'class': 'positive', 'polarity': 0.14, 'subje...
31035    {'class': 'neutral', 'polarity': 0.0, 'subject...
31036    {'class': 'positive', 'polarity': 0.16, 'subje...
Name: sentiment, Length: 31037, dtype: object

In [14]:
df["class"] = sentiment.apply(lambda x: x.get("class"))
df["polarity"] = pd.to_numeric(sentiment.apply(lambda x: x.get("polarity")), errors="coerce")
df["subjectivity"] = pd.to_numeric(sentiment.apply(lambda x: x.get("subjectivity")), errors="coerce")

In [15]:
df.head()

,date,sentiment,source,subject,text,title,url,date_raw,class,polarity,subjectivity
0,2023-12-19 06:40:41,"{'class': 'negative', 'polarity': -0.1, 'subje...",CryptoNews,altcoin,Grayscale CEO Michael Sonnenshein believes the...,Grayscale CEO Calls for Simultaneous Approval ...,https://cryptonews.comhttps://cryptonews.com/n...,2023-12-19 06:40:41,negative,-0.10,0.60
1,2023-12-19 06:03:24,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,"In an exclusive interview with CryptoNews, Man...",Indian Government is Actively Collaborating Wi...,https://cryptonews.comhttps://cryptonews.com/n...,2023-12-19 06:03:24,neutral,0.00,0.00
2,2023-12-19 05:55:14,"{'class': 'positive', 'polarity': 0.05, 'subje...",CryptoNews,blockchain,According to the Federal Court ruling on Decem...,Judge Approves Settlement: Binance to Pay $1.5...,https://cryptonews.comhttps://cryptonews.com/n...,2023-12-19 05:55:14,positive,0.05,0.05
3,2023-12-19 05:35:26,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CoinTelegraph,blockchain,Some suggest EVM inscriptions are the latest w...,Why a gold rush for inscriptions has broken ha...,https://cointelegraph.com/news/inscriptions-ev...,2023-12-19 05:35:26,positive,0.50,0.90
4,2023-12-19 05:31:08,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CoinTelegraph,ethereum,A decision by bloXroute Labs to start censorin...,‘Concerning precedent’ — bloXroute Labs' MEV r...,https://cointelegraph.com/news/concerning-prec...,2023-12-19 05:31:08,neutral,0.00,0.00


In [16]:
df["source"].unique()

array(['CryptoNews', 'CoinTelegraph', 'CryptoPotato'], dtype=object)

In [17]:
df["subject"].unique()

array(['altcoin', 'blockchain', 'ethereum', 'bitcoin', 'nft', 'defi'],
      dtype=object)

In [18]:
df["class"].unique()

array(['negative', 'neutral', 'positive'], dtype=object)

In [19]:
df["polarity"].sort_values().unique()

array([-1.  , -0.88, -0.8 , -0.75, -0.7 , -0.67, -0.65, -0.62, -0.6 ,
       -0.59, -0.57, -0.55, -0.53, -0.52, -0.5 , -0.49, -0.48, -0.47,
       -0.46, -0.45, -0.44, -0.43, -0.42, -0.41, -0.4 , -0.39, -0.38,
       -0.37, -0.36, -0.35, -0.34, -0.33, -0.32, -0.31, -0.3 , -0.29,
       -0.28, -0.27, -0.26, -0.25, -0.24, -0.23, -0.22, -0.21, -0.2 ,
       -0.19, -0.18, -0.17, -0.16, -0.15, -0.14, -0.13, -0.12, -0.11,
       -0.1 , -0.09, -0.08, -0.07, -0.06, -0.05, -0.04, -0.03, -0.02,
       -0.01,  0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,
        0.08,  0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,
        0.17,  0.18,  0.19,  0.2 ,  0.21,  0.22,  0.23,  0.24,  0.25,
        0.26,  0.27,  0.28,  0.29,  0.3 ,  0.31,  0.32,  0.33,  0.34,
        0.35,  0.36,  0.37,  0.38,  0.39,  0.4 ,  0.41,  0.42,  0.43,
        0.44,  0.45,  0.46,  0.47,  0.48,  0.49,  0.5 ,  0.51,  0.52,
        0.53,  0.54,  0.55,  0.56,  0.57,  0.58,  0.59,  0.6 ,  0.61,
        0.62,  0.63,

In [20]:
df["subjectivity"].sort_values().unique()

array([0.  , 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.97, 1.  ])

In [21]:
df['class'].value_counts()

class
positive    13964
neutral     10555
negative     6518
Name: count, dtype: int64

In [22]:
df['class'].value_counts(normalize=True)

class
positive    0.449915
neutral     0.340078
negative    0.210007
Name: proportion, dtype: float64

In [23]:
df["text_len"] = df["text"].str.len()
df.groupby("class")["text_len"].describe()

,count,mean,std,min,25%,50%,75%,max
class,,,,,,,,
negative,6518.0,144.774010,46.940715,39.0,115.0,138.0,164.0,513.0
neutral,10555.0,126.677215,37.921022,27.0,101.0,125.0,148.0,450.0
positive,13964.0,149.925666,52.053271,26.0,118.0,141.0,169.0,508.0


In [24]:
df.duplicated(subset=["text"]).mean()

0.019782839836324388

In [25]:
duplicate_text_rows = df[df.duplicated(subset=["text"], keep=False)]
duplicate_text_rows.sort_values("text")

,date,sentiment,source,subject,text,title,url,date_raw,class,polarity,subjectivity,text_len
6136,2023-08-21 14:16:25,"{'class': 'positive', 'polarity': 0.15, 'subje...",CoinTelegraph,bitcoin,A tale of two types of Bitcoin investor emerge...,Bitcoin speculators are underwater on 88% of t...,https://cointelegraph.com/news/bitcoin-specula...,2023-08-21 14:16:25,positive,0.15,0.42,125
6135,2023-08-21 14:17:11,"{'class': 'positive', 'polarity': 0.15, 'subje...",CoinTelegraph,bitcoin,A tale of two types of Bitcoin investor emerge...,Bitcoin speculators are underwater on 88% of t...,https://cointelegraph.com/news/bitcoin-specula...,2023-08-21 14:17:11,positive,0.15,0.42,125
103,2023-12-16 10:00:15,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CryptoNews,altcoin,A weekly round-up of top crypto news from Cryp...,"Eliminating Fees, Killing Features, Reducing P...",https://cryptonews.comhttps://cryptonews.com/n...,2023-12-16 10:00:15,positive,0.50,0.50,105
482,2023-12-09 10:00:39,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CryptoNews,altcoin,A weekly round-up of top crypto news from Cryp...,"Michael Saylor’s 2BNs, Adam Back’s 700Ks, Jami...",https://cryptonews.comhttps://cryptonews.com/n...,2023-12-09 10:00:39,positive,0.50,0.50,105
4596,2023-09-21 04:58:00,"{'class': 'positive', 'polarity': 0.15, 'subje...",CryptoNews,blockchain,ASIC Takes Legal Action Against Bit Trade Pty ...,ASIC Initiates Civil Proceedings Against Bit T...,https://cryptonews.com/news/asic-initiates-civ...,2023-09-21 04:58:00,positive,0.15,0.15,150
...,...,...,...,...,...,...,...,...,...,...,...,...
30936,2021-11-22 16:23:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,ethereum,"Your daily, bite-sized digest of cryptoasset a...","Investments in BTC Keep Growing, in ETH - Drop...",https://cryptonews.com/news/investments-in-btc...,2021-11-22 16:23:00,neutral,0.00,0.00,73
30958,2021-11-18 16:41:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,ethereum,"Your daily, bite-sized digest of cryptoasset a...","Bipartisan Bill Aims to Fix Infra Bill, TIME H...",https://cryptonews.com/news/bipartisan-bill-ai...,2021-11-18 16:41:00,neutral,0.00,0.00,73
30970,2021-11-17 16:32:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,bitcoin,"Your daily, bite-sized digest of cryptoasset a...","Crypto Gift Plans, Bitcoin Goes Baseball, SEC ...",https://cryptonews.com/news/crypto-gift-plans-...,2021-11-17 16:32:00,neutral,0.00,0.00,73
21541,2022-08-26 21:08:39,"{'class': 'negative', 'polarity': -0.04, 'subj...",CoinTelegraph,ethereum,“Should an ETH PoW fork arise following The Me...,Coinbase says it will 'evaluate any potential ...,https://cointelegraph.com/news/coinbase-says-i...,2022-08-26 21:08:39,negative,-0.04,0.20,165


In [26]:
duplicate_title_rows = df[df.duplicated(subset=["title"], keep=False)]
duplicate_title_rows.sort_values("title")

,date,sentiment,source,subject,text,title,url,date_raw,class,polarity,subjectivity,text_len
4596,2023-09-21 04:58:00,"{'class': 'positive', 'polarity': 0.15, 'subje...",CryptoNews,blockchain,ASIC Takes Legal Action Against Bit Trade Pty ...,ASIC Initiates Civil Proceedings Against Bit T...,https://cryptonews.com/news/asic-initiates-civ...,2023-09-21 04:58:00,positive,0.15,0.15,150
4597,2023-09-21 04:58:00,"{'class': 'positive', 'polarity': 0.15, 'subje...",CryptoNews,blockchain,ASIC Takes Legal Action Against Bit Trade Pty ...,ASIC Initiates Civil Proceedings Against Bit T...,https://cryptonews.com/news/asic-initiates-civ...,2023-09-21 04:58:00,positive,0.15,0.15,150
28745,2022-03-02 10:53:00,"{'class': 'positive', 'polarity': 0.1, 'subjec...",CryptoNews,bitcoin,"Since Jerome Powell's last appearance, the eco...",All Eyes on US Congress as Fed Chair Powell Pr...,https://cryptonews.com/news/all-eyes-us-congre...,2022-03-02 10:53:00,positive,0.10,0.29,111
28744,2022-03-02 10:53:00,"{'class': 'positive', 'polarity': 0.1, 'subjec...",CryptoNews,bitcoin,"Since Jerome Powell's last appearance, the eco...",All Eyes on US Congress as Fed Chair Powell Pr...,https://cryptonews.com/news/all-eyes-on-us-con...,2022-03-02 10:53:00,positive,0.10,0.29,111
2888,2023-10-25 06:26:00,"{'class': 'positive', 'polarity': 0.07, 'subje...",CryptoNews,blockchain,Crypto exchange giant Binance seems to have co...,Binance is Reportedly Behind New Hong Kong Cry...,https://cryptonews.com/news/binance-reportedly...,2023-10-25 06:26:00,positive,0.07,0.73,194
...,...,...,...,...,...,...,...,...,...,...,...,...
7947,2023-07-14 05:39:30,"{'class': 'negative', 'polarity': -0.12, 'subj...",CoinTelegraph,altcoin,MATIC price surged due to favorable regulation...,Why is Polygon (MATIC) price up today?,https://cointelegraph.com/news/why-is-polygon-...,2023-07-14 05:39:30,negative,-0.12,0.38,138
1811,2023-11-14 18:00:43,"{'class': 'positive', 'polarity': 0.25, 'subje...",CryptoNews,nft,Disney hasannounceda partnership with NBA Top ...,announced,https://cryptonews.comhttps://www.dapperlabs.c...,2023-11-14 18:00:43,positive,0.25,0.25,110
2849,2023-10-25 18:05:22,"{'class': 'positive', 'polarity': 0.12, 'subje...",CryptoNews,blockchain,"Bitfinex Securities, a subsidiary of the Bitfi...",announced,https://cryptonews.comhttps://capital.bitfinex...,2023-10-25 18:05:22,positive,0.12,0.54,255
9258,2023-06-16 14:00:13,"{'class': 'positive', 'polarity': 0.2, 'subjec...",CoinTelegraph,ethereum,Ready Games’ software development kit is toute...,dApp Store kit adopts new tech stack to power ...,https://cointelegraph.com/news/dapp-store-kit-...,2023-06-16 14:00:13,positive,0.20,0.50,110


In [27]:
duplicate_text_title = df[df.duplicated(subset=["text", "title"], keep=False)]
duplicate_text_title.sort_values(["title", "text"])

,date,sentiment,source,subject,text,title,url,date_raw,class,polarity,subjectivity,text_len
4596,2023-09-21 04:58:00,"{'class': 'positive', 'polarity': 0.15, 'subje...",CryptoNews,blockchain,ASIC Takes Legal Action Against Bit Trade Pty ...,ASIC Initiates Civil Proceedings Against Bit T...,https://cryptonews.com/news/asic-initiates-civ...,2023-09-21 04:58:00,positive,0.15,0.15,150
4597,2023-09-21 04:58:00,"{'class': 'positive', 'polarity': 0.15, 'subje...",CryptoNews,blockchain,ASIC Takes Legal Action Against Bit Trade Pty ...,ASIC Initiates Civil Proceedings Against Bit T...,https://cryptonews.com/news/asic-initiates-civ...,2023-09-21 04:58:00,positive,0.15,0.15,150
28744,2022-03-02 10:53:00,"{'class': 'positive', 'polarity': 0.1, 'subjec...",CryptoNews,bitcoin,"Since Jerome Powell's last appearance, the eco...",All Eyes on US Congress as Fed Chair Powell Pr...,https://cryptonews.com/news/all-eyes-on-us-con...,2022-03-02 10:53:00,positive,0.10,0.29,111
28745,2022-03-02 10:53:00,"{'class': 'positive', 'polarity': 0.1, 'subjec...",CryptoNews,bitcoin,"Since Jerome Powell's last appearance, the eco...",All Eyes on US Congress as Fed Chair Powell Pr...,https://cryptonews.com/news/all-eyes-us-congre...,2022-03-02 10:53:00,positive,0.10,0.29,111
5945,2023-08-24 15:01:28,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CoinTelegraph,bitcoin,"BTC price support is yet to reappear, with Bit...",Bitcoin 'overconfidence reigns' but bulls must...,https://cointelegraph.com/news/bitcoin-overcon...,2023-08-24 15:01:28,positive,0.50,0.89,98
5949,2023-08-24 14:11:52,"{'class': 'positive', 'polarity': 0.5, 'subjec...",CoinTelegraph,bitcoin,"BTC price support is yet to reappear, with Bit...",Bitcoin 'overconfidence reigns' but bulls must...,https://cointelegraph.com/market-releases/bitc...,2023-08-24 14:11:52,positive,0.50,0.89,98
6172,2023-08-20 12:31:15,"{'class': 'negative', 'polarity': -0.14, 'subj...",CoinTelegraph,bitcoin,Bitcoin is giving plenty of longtime market pa...,Bitcoin on the way to 'bearadise?' $20K is bac...,https://cointelegraph.com/news/bitcoin-bearadi...,2023-08-20 12:31:15,negative,-0.14,0.44,142
6173,2023-08-20 11:35:46,"{'class': 'negative', 'polarity': -0.14, 'subj...",CoinTelegraph,bitcoin,Bitcoin is giving plenty of longtime market pa...,Bitcoin on the way to 'bearadise?' $20K is bac...,https://cointelegraph.com/news/bitcoin-bearadi...,2023-08-20 11:35:46,negative,-0.14,0.44,142
6135,2023-08-21 14:17:11,"{'class': 'positive', 'polarity': 0.15, 'subje...",CoinTelegraph,bitcoin,A tale of two types of Bitcoin investor emerge...,Bitcoin speculators are underwater on 88% of t...,https://cointelegraph.com/news/bitcoin-specula...,2023-08-21 14:17:11,positive,0.15,0.42,125
6136,2023-08-21 14:16:25,"{'class': 'positive', 'polarity': 0.15, 'subje...",CoinTelegraph,bitcoin,A tale of two types of Bitcoin investor emerge...,Bitcoin speculators are underwater on 88% of t...,https://cointelegraph.com/news/bitcoin-specula...,2023-08-21 14:16:25,positive,0.15,0.42,125


In [28]:
df_dedup = (
    df.sort_values("date")
    .drop_duplicates(subset=["text", "title", "class"], keep="first")
    .reset_index(drop=True)
)

In [29]:
removed_df = df.loc[~df.index.isin(df_dedup.index)]
removed_df

,date,sentiment,source,subject,text,title,url,date_raw,class,polarity,subjectivity,text_len
31018,2021-11-10 21:00:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,blockchain,Seoul attempts to create a 'contactless commun...,Seoul to Launch 'Metaverse' Public Service Pla...,https://cryptonews.com/news/seoul-to-launch-me...,2021-11-10 21:00:00,neutral,0.00,0.00,88
31019,2021-11-10 17:37:00,"{'class': 'positive', 'polarity': 0.14, 'subje...",CryptoNews,ethereum,'The high fees are starting to price out certa...,Ethereum Fees Pushed Higher by DEX Trading as ...,https://cryptonews.com/news/ethereum-fees-push...,2021-11-10 17:37:00,positive,0.14,0.35,68
31020,2021-11-10 16:30:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,altcoin,The NYCCoin is set to launch on Thursday.,"NYCCoin, Cambridge on Tezos, Tether on Avalanc...",https://cryptonews.com/news/nyccoin-cambridge-...,2021-11-10 16:30:00,neutral,0.00,0.00,41
31021,2021-11-10 15:37:00,"{'class': 'positive', 'polarity': 0.16, 'subje...",CryptoNews,bitcoin,Bitcoin (BTC) hit yet another all-time high on...,Bitcoin Hits All-Time High on Record US Inflat...,https://cryptonews.com/news/bitcoin-hits-all-t...,2021-11-10 15:37:00,positive,0.16,0.54,56
31022,2021-11-10 13:10:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,bitcoin,Bitcoin price is struggling to gain momentum f...,Bitcoin and Ethereum Prepare For Next Move,https://cryptonews.com/news/bitcoin-and-ethere...,2021-11-10 13:10:00,neutral,0.00,0.00,70
31023,2021-11-10 13:02:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,nft,'Blockchain and NFTs are the future of video g...,72% of Surveyed Game Developers Interested in ...,https://cryptonews.com/news/72-of-surveyed-gam...,2021-11-10 13:02:00,neutral,0.00,0.12,52
31024,2021-11-10 11:52:00,"{'class': 'positive', 'polarity': 0.25, 'subje...",CryptoNews,altcoin,KDA surged higher in the crypto market this mo...,"Kadena Soars Further on DeFi, NFT Gaming News",https://cryptonews.com/news/kadena-soars-furth...,2021-11-10 11:52:00,positive,0.25,0.50,50
31025,2021-11-10 11:09:00,"{'class': 'negative', 'polarity': -0.02, 'subj...",CryptoNews,bitcoin,The much-awaited wallet is scheduled to be lau...,Robinhood Crypto Wallet Waitlist Attracts 1.6M...,https://cryptonews.com/news/robinhood-crypto-w...,2021-11-10 11:09:00,negative,-0.02,0.47,86
31026,2021-11-10 09:47:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",CryptoNews,nft,The artist's record-breaking digital artwork s...,Beeple's 'HUMAN ONE' Physical and Digital Piec...,https://cryptonews.com/news/beeples-human-one-...,2021-11-10 09:47:00,neutral,0.00,0.00,89
31027,2021-11-10 09:08:00,"{'class': 'positive', 'polarity': 0.11, 'subje...",CryptoNews,altcoin,Altcoins have never before represented more th...,"Altcoins Eclipse Combined Bitcoin, Ethereum Tr...",https://cryptonews.com/news/altcoins-eclipse-c...,2021-11-10 09:08:00,positive,0.11,0.47,144


In [30]:
len(removed_df)

19